In [4]:
import matplotlib.pyplot as plt
import os
import numpy as np
import math
import subprocess

In [ ]:
#This cell is not necessary, it just made my outputs cleaner at times 
import os

# Strip out the exported bash functions that break Jupyter subshells
for key in list(os.environ):
    if key.startswith("BASH_FUNC_"):
        del os.environ[key]

print("✅ Cleaned exported bash functions. Subshells will be quiet now.")


✅ Cleaned exported bash functions. Subshells will be quiet now.


In [6]:
# set working directory for all future cells
os.chdir("/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_Apo_Protein_Run")
# and check with bash command pwd - note the exclamation mark at the beginning
!pwd
# or in python
os.getcwd()

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_Apo_Protein_Run


'/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_Apo_Protein_Run'

In [ ]:
#If you have the amber source file you can replace this entire cell with the source file 
# for example just uncomment and run the below line and remove the rest
# !source /sb/apps/amber22/amber.sh
import os, subprocess

def load_amber_env():
    """Load AmberTools + Amber environment into the Jupyter kernel."""
    bash_cmd = """
    module purge
    module load StdEnvACCRE/2023
    module load gcc/12.3 openmpi/4.1.5 cuda/12.2
    module load ambertools/25.0
    module load scipy-stack/2025a
    module load amber/24
    env
    """
    env_output = subprocess.check_output(["bash", "-c", bash_cmd], text=True)
    for line in env_output.splitlines():
        if "=" in line:
            k, v = line.split("=", 1)
            os.environ[k] = v
    print("✅ Amber environment loaded into Jupyter kernel.")


In [10]:
load_amber_env()

The following modules were not unloaded:
  (Use "module --force purge" to unload all):

  1) CCconfig        6)  ucx/1.14.1         11) flexiblas/3.3.1
  2) gentoo/2023     7)  libfabric/1.18.0   12) imkl/2023.2.0
  3) gcccore/.12.3   8)  pmix/4.2.4         13) StdEnvACCRE/2023
  4) gcc/12.3        9)  ucc/1.2.0
  5) hwloc/2.9.1     10) openmpi/4.1.5
Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2023b
(required by: plumed/2.9.0, ambertools/25.0)
-------------------------------------------------------------------------------




The following have been reloaded with a version change:
  1) ipykernel/2023b => ipykernel/2025a
  2) scipy-stack/2023b => scipy-stack/2025a



✅ Amber environment loaded into Jupyter kernel.


Lmod Warning:
-------------------------------------------------------------------------------
The following dependent module(s) are not currently loaded: scipy-stack/2025a
(required by: amber/24)
-------------------------------------------------------------------------------




Inactive Modules:
  1) arrayfire/3.9.0     2) cudnn/8.9.5.29     3) glfw/3.3.8     4) spdlog/1.11.0

Due to MODULEPATH changes, the following have been reloaded:
  1) ambertools/25.0     2) plumed/2.9.0     3) ucc-cuda/1.2.0     4) ucx-cuda/1.14.1

The following have been reloaded with a version change:
  1) cuda/12.2 => cuda/12.6
  2) cudacore/.12.2.2 => cudacore/.12.6.2
  3) ipykernel/2025a => ipykernel/2023b
  4) nccl/2.18.3 => nccl/2.26.2
  5) scipy-stack/2025a => scipy-stack/2023b



In [56]:
%%bash
# Write tleap.in file for building solvated system

cat > tleap.in <<'EOF'
# load script using: tleap -s -f tleap.in

source leaprc.protein.ff14SB
source leaprc.water.tip3p

complex = loadPdb Models/H++_server_output/6ONI_built.replaced.pqr

check complex
charge complex

addions complex Na+ 0
addions complex Cl- 0

solvateoct complex TIP3PBOX 10.0 iso

#addions complex K+ 14
#addions complex Cl- 14

saveamberparm complex prepin.prmtop prepin.inpcrd
savepdb complex prepin.pdb

quit
EOF

echo "✅ tleap.in file written successfully."


bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


✅ tleap.in file written successfully.


In [ ]:
!tleap -s -f tleap.in

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `module'
/bin/sh: ml: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `ml'
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/prep to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat

In [ ]:
#Check the last line of the output this output for the box dimensions 
!tail -n 1 prepin.inpcrd

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
  85.3530531  85.3530531  85.3530531 109.4712190 109.4712190 109.4712190


In [32]:
#perl molarity.perl {conc in M} {pasted coordinates from above} 
#keep 0.05 it tells the perl script that we desire to have 50mM Salt 
#Just copy the box dimensions from the last cell and add it here and run this cell
!perl Scripts/molarity.perl 0.05 85.3530531  85.3530531  85.3530531 109.4712190 109.4712190 109.4712190

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
 MOLARITY =    0.050
 Box size =   85.353   85.353   85.353  109.471  109.471  109.471
 Volume = 478669.016

   14.413 molecules are necessary to make a molarity of   0.05 M



In [ ]:
# The cell above shows that we need to add ~14 molecules of K and Cl to get 50mM,

!sed -i 's/#addions complex K+ 14/addions complex K+ 14/; s/#addions complex Cl- 14/addions complex Cl- 14/; s/addions complex Cl- 0/addions complex Cl- 1/;' tleap.in
!tail 10 tleap.in

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
tail: cannot open '10' for reading: No such file or directory
==> tleap.in <==

solvateoct complex TIP3PBOX 10.0 iso

addions complex K+ 14
addions complex Cl- 14

saveamberparm complex prepin.prmtop prepin.inpcrd
savepdb complex prepin.pdb

quit


In [34]:
#Rerun tleap to get the updates input files with added ions 
!tleap -s -f tleap.in

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `module'
/bin/sh: ml: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `ml'
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/prep to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/lib to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat/leap/parm to search path.
-I: Adding /cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/MPI/gcc12/openmpi4/ambertools/25.0/dat

In [ ]:
#Write the minimization files in a new directory (keeps the flies organized) 
!mkdir Minimization
%cd Minimization 

!sh ../Scripts/mesmy.sh > log.mesmy

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_U

In [ ]:
%%bash
# Create a ParmEd input file with HMR commands this method effectively speeds up the simulations 
cat > hmr_commands.in <<EOF
HMassRepartition
outparm prepinHMR.prmtop
go
quit
EOF

# Run ParmEd with the input file
parmed ../prepin.prmtop < hmr_commands.in


bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'



  ____    .---.  ,-----.  _____            
.'    `,.':   :'' :    :.'     `.          
| :   :    ' '     '  '   :   :  \         
|  ' '                     ' '    \        
|       v                          \       
|       |       v                   \      
|       |       |       v            \     
|   P   |   A   |   R   |   M   |     \    
|       |       |       |       |      \   
|       |       |       |       |       \  
|       |       |       |       |        \ 
| ===== |       |       |       |         )
 \ --- /| ===== |       |       |        / 
  `---'  \ --- /| ===== | ===== |       /  
          `---'  \ ---- A ----- /      /   
                  `----'.`-----'      /    
                       /             /     
                       '-._    ED   /      
                           '.      /       
                             `----' 

ParmEd: a Parameter file Editor


Loaded Amber topology file ../prepin.prmtop
Reading input from STDIN...
> Repartitioning hy

In [ ]:
#Copy the requried files, take a look at these scripts that will be used 
#production.pl is the driver script that will run the simulations in chunks 
#prod1.in.start is used for the first 100ns and tell amber to sample initial velocities friom boltzmann distribution
#prod1.in is used for the rest of the simulations and it reads the last frame of the previous 100ns simulation and continues from there
%cd ../
!mkdir Production
%cd Production
!cp ../Scripts/production.pl .
!cp ../Scripts/prod1.in.start .
!cp ../Scripts/prod1.in .

/panfs/accrepfs.vampire/data/kojetin_lab/Mithun/My_Github/Amber_Apo_Protein_Run/Production
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'


In [54]:
#Test to see if it works 
!perl production.pl > PRODUCTION.OUT

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
^C


In [58]:
%%bash
# Write slrum script in the working directory, later submit to ACCRE 

cat > 6ONI_submit_slurm <<'EOF'
#!/bin/bash
#SBATCH --job-name=Amber_Apo_6ONI_Run
#SBATCH --mail-type=ALL
#SBATCH --nodes=1
#SBATCH --mem=40G
#SBATCH --time=5-00:00:00
#SBATCH --partition=batch_gpu
#SBATCH --account=csb_gpu_acc
#SBATCH --gres=gpu:nvidia_titan_x:1   # request exactly 3 GPUs on this node
#SBATCH --output=PPARg_meta_out_%j.log
#SBATCH --error=PPARg_meta_err_%j.log

module purge
module load StdEnvACCRE/2023
module load gcc/12.3 openmpi/4.1.5 cuda/12.2
module load ambertools/25.0
module load scipy-stack/2025a
module load amber/24
    
perl production.pl > PRODUCTION.OUT

EOF

bash: ml: line 1: syntax error: unexpected end of file
bash: error importing function definition for `ml'
bash: module: line 1: syntax error: unexpected end of file
bash: error importing function definition for `module'


In [59]:
!ls

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
6ONI_submit_slurm  md_prod1.mdinfo  prod1.in.start  production.pl  PTR_002.out
md_prod1_001.nc    prod1.in	    PRODUCTION.OUT  PTR_001.out


In [60]:
!sbatch 6ONI_submit_slurm

/bin/bash: ml: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `ml'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `module'
Submitted batch job 5155533
